### CNN Model Training with CIFAR10 - Dataset Load & Setting

In [13]:
import torch
from torchvision import datasets as D
from torchvision import transforms as T
from torch.utils.data import random_split

transform_train = T.Compose([
    T.RandomCrop(32, padding=4), # random한 위치에서 crop 진행(32 size, 모든 경계에 padding)
    T.RandomHorizontalFlip(), 
    T.RandomRotation(15), # -15~15도 내에서 회전
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), # hue: 0~255의 컬러 #saturarion: hue의 순수도 contrast: 명암
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = T.Compose([
    T.ToTensor(),
    T.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

batch_size = 128 # 이미지 묶음 당 이미지 개수

#CIFAR10 - 10개 클래스, 32x32x3 이미지

#train data 정의
trainset = D.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train) 

# trainloader를 통해 train_data를 load
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True)

#test data 정의
testset = D.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
train_dataset, val_dataset = random_split(trainset, [train_size, val_size])
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


### Pretrained model - VGG16bn

In [14]:
from torchvision import models
import torch.nn as nn
vgg16_bn_model = models.vgg16_bn(pretrained = True)
vgg16_bn_model.classifier[6] = nn.Linear(in_features=4096, out_features=10)

### Training 과정

In [15]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

class EarlyStopping:
    def __init__(self, patience=10, verbose=False):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0
            
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = vgg16_bn_model.to(device)
def training(model, trainloader):
    num_epoch = 200
    total_step = len(trainloader)
    epoch_losses = []
    learning_rate = 1e-4
    momentum_value = 0.9
    weight_decay_value = 1e-4
    patience = 20
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    criterion = nn.CrossEntropyLoss() #loss function 설정
    optimizer = optim.Adam(vgg16_bn_model.parameters(), lr=learning_rate, weight_decay = weight_decay_value) 
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(trainloader), epochs=100)

    for epoch in range(num_epoch):
        model.train()
        running_loss = 0.0
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(trainloader)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        print(f'Epoch {epoch+1}/{num_epoch}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

        scheduler.step(val_loss)
        early_stopping(val_loss)

        if early_stopping.early_stop:
            print("Early stopping")
            break



In [16]:
training(model, trainloader)

Epoch 1/100, Train Loss: 1.0191, Val Loss: 0.7110
Epoch 2/100, Train Loss: 0.6852, Val Loss: 0.5862
Epoch 3/100, Train Loss: 0.5976, Val Loss: 0.5356
Epoch 4/100, Train Loss: 0.5309, Val Loss: 0.4607
Epoch 5/100, Train Loss: 0.4838, Val Loss: 0.4242
Epoch 6/100, Train Loss: 0.4558, Val Loss: 0.4516
EarlyStopping counter: 1 out of 20
Epoch 7/100, Train Loss: 0.4388, Val Loss: 0.4449
EarlyStopping counter: 2 out of 20
Epoch 8/100, Train Loss: 0.4144, Val Loss: 0.3505
Epoch 9/100, Train Loss: 0.3995, Val Loss: 0.3774
EarlyStopping counter: 1 out of 20
Epoch 10/100, Train Loss: 0.4013, Val Loss: 0.4129
EarlyStopping counter: 2 out of 20
Epoch 11/100, Train Loss: 0.4003, Val Loss: 0.3224
Epoch 12/100, Train Loss: 0.3455, Val Loss: 0.3764
EarlyStopping counter: 1 out of 20
Epoch 13/100, Train Loss: 0.6343, Val Loss: 0.4270
EarlyStopping counter: 2 out of 20
Epoch 14/100, Train Loss: 0.4036, Val Loss: 0.3492
EarlyStopping counter: 3 out of 20
Epoch 15/100, Train Loss: 0.3362, Val Loss: 0.2721

### CNN model Inference

In [17]:
def inference(model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        
        for inputs, labels in testloader:
            images, labels = inputs.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the test images: {100 * correct // total:.4f} %')

In [18]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model=cnn_model.to(device)
inference(model)

Accuracy of the network on the test images: 91.0000 %
